In [1]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [16]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_sub = pd.read_csv("submit.csv", header=None)

In [17]:
# trainデータの確認
print(f"train shape: {train.shape}")
train.head(3)

train shape: (242156, 10)


,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,88194295,1800.0,3 years,14.49,C4,NaN,debt_consolidation,665.0,Individual,FullyPaid
1,5146039,1200.0,5 years,16.29,C4,2 years,debt_consolidation,700.0,Individual,ChargedOff
2,3095896,2000.0,5 years,21.98,E4,10 years,home_improvement,670.0,Individual,FullyPaid


In [18]:
# testデータの確認
print(f"test shape: {test.shape}")
test.head(3)

test shape: (26906, 9)


,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type
0,1496754,1912.5,3 years,10.16,B1,5 years,debt_consolidation,725.0,Individual
1,84909594,1800.0,3 years,8.99,B1,9 years,credit_card,695.0,Individual
2,1165403,550.0,3 years,14.65,C2,10 years,credit_card,660.0,Individual


In [19]:
train["loan_status"].value_counts()

loan_status
FullyPaid     193815
ChargedOff     48341
Name: count, dtype: int64

In [20]:
train["loan_status"].replace({"FullyPaid":0,"ChargedOff":1}, inplace=True)

In [21]:
def concat_df(train_data, test_data):
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

df_all = concat_df(train, test)

In [22]:
df_all.head(5)

,application_type,credit_score,employment_length,grade,id,interest_rate,loan_amnt,loan_status,purpose,term
0,Individual,665.0,NaN,C4,88194295,14.49,1800.0,0.0,debt_consolidation,3 years
1,Individual,700.0,2 years,C4,5146039,16.29,1200.0,1.0,debt_consolidation,5 years
2,Individual,670.0,10 years,E4,3095896,21.98,2000.0,0.0,home_improvement,5 years
3,Individual,710.0,4 years,A5,88625044,8.59,1000.0,0.0,debt_consolidation,3 years
4,Individual,680.0,4 years,C1,1178189,13.99,1500.0,0.0,debt_consolidation,3 years


In [23]:
df_all.dtypes

application_type      object
credit_score         float64
employment_length     object
grade                 object
id                     int64
interest_rate        float64
loan_amnt            float64
loan_status          float64
purpose               object
term                  object
dtype: object

In [24]:
df_all["application_type"].value_counts()

application_type
Individual    262329
Joint App       6733
Name: count, dtype: int64

In [31]:
df_all_app = pd.get_dummies(df_all["application_type"], prefix="app-type")
df_all = df_all.reset_index().merge(df_all_app, left_index=True, right_index=True).drop('index', axis=1)


In [46]:
df_all.dtypes

credit_score           float64
employment_length       object
id                       int64
interest_rate          float64
loan_amnt              float64
loan_status            float64
purpose                 object
term                     int64
app-type_Individual      int64
app-type_Joint App       int64
dtype: object

In [37]:
list = ["app-type_Individual","app-type_Joint App"]
for column in list:
    df_all[column].replace({True:1, False:0}, inplace=True)
df_all

,application_type,credit_score,employment_length,grade,id,interest_rate,loan_amnt,loan_status,purpose,term,app-type_Individual,app-type_Joint App
0,Individual,665.0,NaN,C4,88194295,14.49,1800.0,0.0,debt_consolidation,3 years,1,0
1,Individual,700.0,2 years,C4,5146039,16.29,1200.0,1.0,debt_consolidation,5 years,1,0
2,Individual,670.0,10 years,E4,3095896,21.98,2000.0,0.0,home_improvement,5 years,1,0
3,Individual,710.0,4 years,A5,88625044,8.59,1000.0,0.0,debt_consolidation,3 years,1,0
4,Individual,680.0,4 years,C1,1178189,13.99,1500.0,0.0,debt_consolidation,3 years,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
269057,Individual,735.0,5 years,A4,5064630,7.90,800.0,NaN,debt_consolidation,3 years,1,0
269058,Individual,715.0,8 years,A4,94416733,7.49,750.0,NaN,debt_consolidation,3 years,1,0
269059,Individual,695.0,7 years,C4,120604638,15.05,840.0,NaN,car,3 years,1,0
269060,Individual,680.0,0 years,C2,1110314,14.27,1000.0,NaN,debt_consolidation,3 years,1,0


In [41]:
df_all.drop('grade',axis=1, inplace=True)
df_all.drop('application_type',axis=1, inplace=True)

In [42]:
df_all['term'].value_counts()

term
3 years    209582
5 years     59480
Name: count, dtype: int64

In [43]:
df_all["term"].replace({"3 years":3, "5 years":5}, inplace=True)

In [44]:
df_all["employment_length"].value_counts()

employment_length
10 years    87812
2 years     24496
3 years     20869
0 years     20666
5 years     18862
1 year      17715
4 years     16519
6 years     14588
7 years     11857
8 years     11031
9 years     10022
Name: count, dtype: int64

In [54]:
df_all["employment_length"].fillna("5 years", inplace=True)
df_all["employment_length"].replace({"0 years":0, "1 year":1, "2 years":2, "3 years":3, "4 years":4, "5 years":5, "6 years":6, "7 years":7, "8 years":8, "9 years":9, "10 years":10},inplace=True)

In [47]:
df_all["purpose"].value_counts()

purpose
debt_consolidation    156752
credit_card            53720
home_improvement       17867
other                  17107
major_purchase          6373
small_business          3718
car                     3228
medical                 3161
moving                  1965
house                   1930
vacation                1884
wedding                 1127
renewable_energy         230
Name: count, dtype: int64

In [48]:
df_all_pur = pd.get_dummies(df_all["purpose"], prefix="purpose")
df_all = df_all.reset_index().merge(df_all_pur, left_index=True, right_index=True).drop('index', axis=1)

In [49]:
df_all.dtypes

credit_score                  float64
employment_length              object
id                              int64
interest_rate                 float64
loan_amnt                     float64
loan_status                   float64
purpose                        object
term                            int64
app-type_Individual             int64
app-type_Joint App              int64
purpose_car                      bool
purpose_credit_card              bool
purpose_debt_consolidation       bool
purpose_home_improvement         bool
purpose_house                    bool
purpose_major_purchase           bool
purpose_medical                  bool
purpose_moving                   bool
purpose_other                    bool
purpose_renewable_energy         bool
purpose_small_business           bool
purpose_vacation                 bool
purpose_wedding                  bool
dtype: object

In [50]:
list = ["purpose_car","purpose_credit_card","purpose_debt_consolidation","purpose_home_improvement",
        "purpose_house","purpose_major_purchase","purpose_medical","purpose_moving",
        "purpose_other","purpose_renewable_energy","purpose_small_business","purpose_vacation","purpose_wedding"]
for column in list:
    df_all[column].replace({True:1, False:0}, inplace=True)
df_all

,credit_score,employment_length,id,interest_rate,loan_amnt,loan_status,purpose,term,app-type_Individual,app-type_Joint App,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,665.0,5 years,88194295,14.49,1800.0,0.0,debt_consolidation,3,1,0,...,0,0,0,0,0,0,0,0,0,0
1,700.0,2 years,5146039,16.29,1200.0,1.0,debt_consolidation,5,1,0,...,0,0,0,0,0,0,0,0,0,0
2,670.0,10 years,3095896,21.98,2000.0,0.0,home_improvement,5,1,0,...,1,0,0,0,0,0,0,0,0,0
3,710.0,4 years,88625044,8.59,1000.0,0.0,debt_consolidation,3,1,0,...,0,0,0,0,0,0,0,0,0,0
4,680.0,4 years,1178189,13.99,1500.0,0.0,debt_consolidation,3,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269057,735.0,5 years,5064630,7.90,800.0,NaN,debt_consolidation,3,1,0,...,0,0,0,0,0,0,0,0,0,0
269058,715.0,8 years,94416733,7.49,750.0,NaN,debt_consolidation,3,1,0,...,0,0,0,0,0,0,0,0,0,0
269059,695.0,7 years,120604638,15.05,840.0,NaN,car,3,1,0,...,0,0,0,0,0,0,0,0,0,0
269060,680.0,0 years,1110314,14.27,1000.0,NaN,debt_consolidation,3,1,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df_all.drop('purpose',axis=1, inplace=True)

In [55]:
df_all.dtypes

credit_score                  float64
employment_length               int64
id                              int64
interest_rate                 float64
loan_amnt                     float64
loan_status                   float64
term                            int64
app-type_Individual             int64
app-type_Joint App              int64
purpose_car                     int64
purpose_credit_card             int64
purpose_debt_consolidation      int64
purpose_home_improvement        int64
purpose_house                   int64
purpose_major_purchase          int64
purpose_medical                 int64
purpose_moving                  int64
purpose_other                   int64
purpose_renewable_energy        int64
purpose_small_business          int64
purpose_vacation                int64
purpose_wedding                 int64
dtype: object

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [57]:
df_train = df_all[:242156]
df_test = df_all[242156:]

In [58]:
X = df_train.drop("loan_status", axis=1).values
y = df_train["loan_status"].values
X_test = df_test.drop("loan_status", axis=1).values

In [59]:
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)
X_test_std = sc.transform(X_test)

In [60]:
# 訓練データと評価データに分割
X_train, X_valid, y_train, y_valid = train_test_split(X_std, y, test_size=0.3, stratify=y, random_state=0)

In [61]:
# LogisticRegression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)

lr_train_pred = lr.predict_proba(X_train)[:, 1]
lr_valid_pred = lr.predict_proba(X_valid)[:, 1]
print(f"Train Score: {roc_auc_score(y_train, lr_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, lr_valid_pred)}")

Train Score: 0.6895288910218642
Valid Score: 0.6837460204130412


In [62]:
# MLPClassifier
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=0)
mlp.fit(X_train, y_train)

mlp_train_pred = mlp.predict_proba(X_train)[:, 1]
mlp_valid_pred = mlp.predict_proba(X_valid)[:, 1]
print(f"Train Score: {roc_auc_score(y_train, mlp_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, mlp_valid_pred)}")

Train Score: 0.7031136111419587
Valid Score: 0.6914807045511877


In [63]:
# XGBClassifier
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=0)
xgb.fit(X_train, y_train)

xgb_train_pred = xgb.predict_proba(X_train)[:, 1]
xgb_valid_pred = xgb.predict_proba(X_valid)[:, 1]
print(f"Train Score: {roc_auc_score(y_train, xgb_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, xgb_valid_pred)}")

Train Score: 0.7609444683247057
Valid Score: 0.6997959485698841


In [71]:
# LGBMClassifier
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=0)
lgb.fit(X_train, y_train)

lgb_train_pred = lgb.predict_proba(X_train)[:, 1]
lgb_valid_pred = lgb.predict_proba(X_valid)[:, 1]
print(f"Train Score: {roc_auc_score(y_train, lgb_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, lgb_valid_pred)}")

ModuleNotFoundError: No module named 'lightgbm'

In [72]:
train_pred = (lr_train_pred + mlp_train_pred +  xgb_train_pred)/3
valid_pred = (lr_valid_pred + mlp_valid_pred + xgb_valid_pred)/3

print(f"Train Score: {roc_auc_score(y_train, train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, valid_pred)}")

Train Score: 0.7295834227640577
Valid Score: 0.6992706910622805


In [74]:
pred = xgb.predict_proba(X_test_std)[:, 1]
for i in range(len(pred)):
  if pred[i] > 0.5:
    pred[i] = 1
  else:
    pred[i] = 0

In [75]:
sample_sub[1] = pred
sample_sub

,0,1
0,1496754,0.0
1,84909594,0.0
2,1165403,0.0
3,91354446,0.0
4,85636932,0.0
...,...,...
26901,5064630,0.0
26902,94416733,0.0
26903,120604638,0.0
26904,1110314,0.0


In [79]:
sample_sub[1] = sample_sub[1].astype(int)

In [77]:
sample_sub.to_csv('submission.csv',index=False,header=None)